In [ ]:
%pip install chardet IPython

In [ ]:
import pandas as pd
import os
import glob
import chardet

In [171]:
if os.path.exists('df.csv'):
    df = pd.read_csv('df.csv')
else:
    dfs = []
    for file in glob.glob(os.path.join('./data', '*.csv')):
        with open(file, 'rb') as f:
            result = chardet.detect(f.read())
            encoding = result['encoding']
        df = pd.read_csv(file, encoding=encoding)
        dfs.append(df)
    df = pd.concat(dfs)
    df = df[df['tags'] != '[]']
    df['prompt'] = ""

In [172]:
df.head()

,name,author,img,license,downloads,tags,prompt
0,W.J. Pearce,Listemageren,https://see.fontimg.com/api/renderfont4/JO0x/e...,Postcardware,3956,"['Poster', 'Hollow', 'Arts & Crafts', 'All Caps']",NaN
1,Wabbit Sans,Chequered Ink,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...","Freeware, Non-Commercial",2541,"['Retro', '70s', 'Groovy']",NaN
2,Wabroye,gluk,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",SIL Open Font License (OFL),24559,"['Fancy', 'Serif', 'Vintage', 'Medieval', 'Dec...",NaN
3,Wacamóler Caps,deFharo,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...","Freeware, Non-Commercial",7083,"['Decorative', 'Movies', 'Old Western', 'Wild ...",NaN
4,WaChaKa,AdamAnt Designs,https://see.fontimg.com/api/renderfont4/jWJR/e...,Freeware,1646,"['All Caps', 'Display']",NaN


### Prompt Generation
The following cells aid in augmenting the dataset with faux prompts that mimic what a user might ask for. Prompts need both adjectives (to correlate with the tags) and nouns (embed adjectives w/ scenarios a user might need a font for). 
Example: 'I need an edgy font for a punk band.'

In [173]:
import random
from IPython.display import Image, display, clear_output

def get_random_row(df):
    random_index = random.randint(0, len(df) - 1)
    if df.iloc[random_index]['prompt'] == '':
        return df.iloc[random_index]
    else:
        get_random_row(df)


In [165]:
random_row = get_random_row(df)
print(random_row['name'], random_row['tags'])

if random_row['img'].startswith('https://'):
    display(Image(url=random_row['img']))

prompt = input("Prompt: ")
random_row['prompt'] = prompt

df.iloc[random_row.name] = random_row
print(df.iloc[random_row.name])

Decondor ['Serif', 'Ligatures', 'Display', 'Narrow']
name                                                  Decondor
author                                             Alit Design
img          data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...
license                               Freeware, Non-Commercial
downloads                                                 3461
tags               ['Serif', 'Ligatures', 'Display', 'Narrow']
prompt       I'd like a trendy yet old feeling font for a d...
Name: 854, dtype: object


/var/folders/mv/1d7qtrsj33g313mpvxzzv5fh0000gn/T/ipykernel_45134/1598917740.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_row['prompt'] = prompt


### Save DataFrame
We need ~400 prompts, and this is a low amount. Since I'm doing this manually, I need to save and re-load the DataFrame. I do that below.

In [167]:
df.to_csv('df.csv', index=False)

In [174]:
# Check how many prompts we have
not_empty_prompts = df[df['prompt'].notna() & (df['prompt'] != '')]
num_not_empty_prompts = len(not_empty_prompts)
print(num_not_empty_prompts)

90
